In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45080
2,Huelva,Confirmados PDIA 14 días,412
3,Huelva,Tasa PDIA 14 días,"80,28528557787868"
4,Huelva,Confirmados PDIA 7 días,179
5,Huelva,Tasa PDIA 7 dias,"34,88122844281622"
6,Huelva,Total Confirmados,45281
7,Huelva,Curados,42715
8,Huelva,Fallecidos,411


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45080.0


/tmp/ipykernel_3640/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12470.0


/tmp/ipykernel_3640/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3640/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3640/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3640/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3425 personas en los últimos 7 días 

Un positivo PCR cada 1776 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45080.0,179.0,412.0,513170.0,34.881228,80.285286,36.0
Huelva-Costa,26690.0,114.0,291.0,289548.0,39.371710,100.501471,27.0
Condado-Campiña,13983.0,54.0,83.0,156231.0,34.564203,53.126460,8.0
Huelva (capital),12470.0,42.0,81.0,143837.0,29.199719,56.313744,7.0
Ayamonte,2048.0,17.0,48.0,21104.0,80.553450,227.445034,6.0
Lepe,2991.0,13.0,29.0,27880.0,46.628407,104.017217,5.0
Paterna del Campo,205.0,12.0,12.0,3457.0,347.121782,347.121782,4.0
Isla Cristina,3107.0,12.0,42.0,21393.0,56.093115,196.325901,3.0
Aljaraque,1614.0,3.0,5.0,21474.0,13.970383,23.283971,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,33.0,1.0,2.0,396.0,252.525253,505.050505,0.0
Cartaya,1950.0,23.0,70.0,20083.0,114.524722,348.553503,2.0
Paterna del Campo,205.0,12.0,12.0,3457.0,347.121782,347.121782,4.0
Higuera de la Sierra,66.0,2.0,4.0,1291.0,154.918668,309.837335,1.0
Santa Bárbara de Casa,46.0,2.0,3.0,1043.0,191.754554,287.631831,1.0
Aroche,128.0,1.0,8.0,3054.0,32.743942,261.951539,0.0
Cumbres Mayores,135.0,1.0,4.0,1749.0,57.175529,228.702115,0.0
Ayamonte,2048.0,17.0,48.0,21104.0,80.553450,227.445034,6.0
Isla Cristina,3107.0,12.0,42.0,21393.0,56.093115,196.325901,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,128.0,1.0,8.0,3054.0,32.743942,261.951539,0.0,0.125000
Punta Umbría,1256.0,1.0,5.0,15355.0,6.512537,32.562683,0.0,0.200000
Cumbres Mayores,135.0,1.0,4.0,1749.0,57.175529,228.702115,0.0,0.250000
Isla Cristina,3107.0,12.0,42.0,21393.0,56.093115,196.325901,3.0,0.285714
Sierra de Huelva-Andévalo Central,3985.0,11.0,34.0,67391.0,16.322654,50.451841,1.0,0.323529
Cartaya,1950.0,23.0,70.0,20083.0,114.524722,348.553503,2.0,0.328571
Minas de Riotinto,178.0,2.0,6.0,3812.0,52.465897,157.397692,0.0,0.333333
Ayamonte,2048.0,17.0,48.0,21104.0,80.553450,227.445034,6.0,0.354167
Huelva-Costa,26690.0,114.0,291.0,289548.0,39.371710,100.501471,27.0,0.391753
